## caching

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache
# chat = ChatOpenAI(temperature=0.1)

# set_llm_cache(InMemoryCache())
set_llm_cache(SQLiteCache("cache.db"))
set_debug(True)
# chat = ChatOpenAI(
#     temperature = 0.1,
#     # streaming = True,
#     # callbacks = [StreamingStdOutCallbackHandler()]
# )

# chat.predict("how do you make italian pasta")

## Serialization


In [ ]:
from langchain.callbacks import get_openai_callback

chat = ChatOpenAI(
    temperature = 0.1,
    # streaming = True,
    # callbacks = [StreamingStdOutCallbackHandler()]
)

with get_openai_callback() as usage:
    a = chat.predict("what is the recipe for soju? explain briefly.")
    b = chat.predict("what is the recipe for bread? explain briefly.")
    print(a,b,"\n")
    # print(usage)
    print()
